In [ ]:
import psycopg2
import os
import django

True

## Migrate old Flask blog DB

In [ ]:
uri1=""
flask_db_conn = psycopg2.connect(uri1)
flask_cursor = flask_db_conn.cursor()

flask_cursor.execute('SELECT id, title, date_posted, content, user_id, topic FROM post')
posts = flask_cursor.fetchall()

(2, 'test4', datetime.datetime(2024, 10, 31, 14, 16, 13, 191352), 'add this post from web, which is deployed on render', 1, 'default')
(4, 'test6', datetime.datetime(2024, 11, 1, 10, 27, 24, 860906), 'Post as Leonard', 2, 'default')
(5, 'Developer', datetime.datetime(2024, 11, 4, 12, 6, 51, 727483), 'I have always liked building things. Flask makes it possible to build a interactive website with my first programming language python. Meanwhile it would be a nice place for someone introverse like me to post some ideas about different topics.', 2, 'default')
(10, 'test1', datetime.datetime(2024, 11, 8, 16, 45, 23, 80999), 'test1', 1, 'tech')
(11, 'test', datetime.datetime(2024, 11, 8, 16, 45, 59, 424530), 'test', 1, 'politics')
(12, 'X', datetime.datetime(2024, 11, 27, 14, 5, 36, 857827), 'What is the common part between Elon Musk and me? We both have our own blog :)', 2, 'personal')
(13, 'Silvester', datetime.datetime(2025, 1, 1, 20, 33, 55, 474895), 'Silvester in Berlin ist hässlich, wi

In [ ]:
os.environ.setdefault('DJANGO_SETTINGS_MODULE', "project.settings")
django.setup()
from django.contrib.auth.models import User
from app.models import Post

In [ ]:
from asgiref.sync import sync_to_async

@sync_to_async
def get_migrator_user():
    return User.objects.get(username='migrator')

migrator_user = await get_migrator_user()

app_posts = []
for row in posts:
    app_post = Post(
        title=row[1],
        content=row[3],
        date_posted=row[2],
        author=migrator_user,
        topic=row[5]
    )
    app_posts.append(app_post)
    
@sync_to_async
def save_posts(posts):
    Post.objects.bulk_create(posts)

await save_posts(app_posts)

In [11]:
dct_topic = {
    'politics': 'Politics',
    'tech': 'Technology',
    'default': 'Default',
    'personal': 'Daily'
}
@sync_to_async
def rename_topics(old_topic, new_topic):
    Post.objects.filter(topic=old_topic).update(topic=new_topic)
    
for old_topic, new_topic in dct_topic.items():
    await rename_topics(old_topic, new_topic)

In [12]:
# Close connection
flask_cursor.close()
flask_db_conn.close()

## Migrate postgresql to sqlite

In [ ]:
python manage.py dumpdata auth.user --output=users.json
python manage.py dumpdata app.Post --output=posts.json
# change setting DB
python manage.py migrate
python manage.py loaddata users.json
python manage.py loaddata posts.json

